In [1]:
from messis.messis import Messis

In [2]:
model = Messis.from_pretrained('crop-classification/messis')

Loaded pretrained weights from './prithvi/models/Prithvi_100M.pt' with partial matching.


In [ ]:
model.forward()